In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_table('cer_2012/File1.txt',header=None,sep=' ')
data2 = pd.read_table('cer_2012/File2.txt',header=None,sep=' ')
data3 = pd.read_table('cer_2012/File3.txt',header=None,sep=' ')
data4 = pd.read_table('cer_2012/File4.txt',header=None,sep=' ')
data5 = pd.read_table('cer_2012/File5.txt',header=None,sep=' ')
data6 = pd.read_table('cer_2012/File6.txt',header=None,sep=' ')

In [3]:
data = pd.concat([data1,data2,data3,data4,data5,data6])

data.columns = ['uid','timestamp','elec']

In [4]:
data

,uid,timestamp,elec
0,1392,19503,0.140
1,1392,19504,0.138
2,1392,19505,0.140
3,1392,19506,0.145
4,1392,19507,0.145
...,...,...,...
35436007,6269,73044,0.425
35436008,6269,73045,0.385
35436009,6269,73046,0.431
35436010,6269,73047,0.459


In [7]:
suervey_pre = pd.read_csv('cer_2012/CER_Electricity_Data/survery_data/Smart meters Residential pre-trial survey data.csv',
                          encoding='gbk')

/tmp/ipykernel_27600/3584262559.py:1: DtypeWarning: Columns (70,71,72,73,74,75,76,77,78,79,90,91,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  suervey_pre = pd.read_csv('cer_2012/CER_Electricity_Data/survery_data/Smart meters Residential pre-trial survey data.csv',


In [9]:
data.uid = data.uid.astype(int)

In [10]:
data = data[data['uid'].isin(suervey_pre['ID'])]
data = data[data['uid'].isin(data['uid'].value_counts()[data['uid'].value_counts()>20000].index.tolist())]

In [12]:
data['uid'].nunique()

4064

In [13]:
import pickle

In [15]:
all_uids = data['uid'].unique()
all_timestamps = np.sort(data['timestamp'].unique())
new_index = pd.MultiIndex.from_product([all_uids, all_timestamps], names=['uid', 'timestamp'])
index_data = pd.DataFrame(index=new_index).reset_index()

In [16]:
index_data['timestamp_rank'] = index_data.groupby('uid')['timestamp'].rank().astype(int)

In [18]:
# 合并原始数据和新DataFrame
merged_data = pd.merge(index_data, data, on=['uid', 'timestamp'], how='left')
merged_data.elec = merged_data.elec.fillna(0)

In [45]:
import random
all_data = []
test_data = []
end_timestamp = random.randint(0,99)
for i in range(5):
    start_timestamp = end_timestamp+1
    end_timestamp = start_timestamp + 96
    print ('start_timestamp:',start_timestamp,' end_timestamp:',end_timestamp)

    load_forecasting_df = merged_data.groupby('uid').first().reset_index()
    elec_data = merged_data[(merged_data['timestamp_rank'] >= start_timestamp) & (merged_data['timestamp_rank'] < end_timestamp)]

    train_data = elec_data.groupby('uid')['elec'].apply(list).reset_index()
    train_data.columns = ['uid','train_elec']

    load_forecasting_df = load_forecasting_df.merge(train_data,on='uid',how='left')

    load_forecasting_df = load_forecasting_df[['uid','train_elec']]
    all_data.append(load_forecasting_df)
    
for i in range(2):
    start_timestamp = end_timestamp+1
    end_timestamp = start_timestamp + 96
    print ('start_timestamp:',start_timestamp,' end_timestamp:',end_timestamp)

    load_forecasting_df = merged_data.groupby('uid').first().reset_index()
    elec_data = merged_data[(merged_data['timestamp_rank'] >= start_timestamp) & (merged_data['timestamp_rank'] < end_timestamp)]

    train_data = elec_data.groupby('uid')['elec'].apply(list).reset_index()
    train_data.columns = ['uid','train_elec']

    load_forecasting_df = load_forecasting_df.merge(train_data,on='uid',how='left')

    load_forecasting_df = load_forecasting_df[['uid','train_elec']]
    test_data.append(load_forecasting_df)

start_timestamp: 94  end_timestamp: 190


start_timestamp: 191  end_timestamp: 287
start_timestamp: 288  end_timestamp: 384
start_timestamp: 385  end_timestamp: 481
start_timestamp: 482  end_timestamp: 578
start_timestamp: 579  end_timestamp: 675
start_timestamp: 676  end_timestamp: 772


In [46]:
all_data = pd.concat(all_data,axis=0).reset_index(drop=True)
test_data = pd.concat(test_data,axis=0).reset_index(drop=True)

In [47]:
final_data = np.array(all_data['train_elec'].tolist())
test_data = np.array(test_data['train_elec'].tolist())

In [52]:
pickle.dump(final_data, open('data_96_train.pkl','wb'))
pickle.dump(test_data, open('data_96_test.pkl','wb'))